# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [16]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [17]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'

#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions).

In [38]:
import urllib.request
uf = urllib.request.urlopen(url)
response = uf.read().decode('utf-8')
results = json.loads(response)
flattened_data = json_normalize(results)
flattened_data1 = json_normalize(flattened_data.articles[0])
flattened_data1


,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,url_key
0,adidas Performance,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -20...",False,[{'path': 'AD/54/3E/0Q/6Q/11/AD543E0Q6-Q11@4.j...,TRAINING EQUIPMENT 3 STREIFEN - Collants - bla...,False,True,True,False,"27,95 €","22,35 €",clothing,"[4-5a, 5-6a, 6-7a, 7-8a, 9-10a, 11-12a, 12-13a...",AD543E0Q6-Q11,adidas-performance-collants-blackwhite-ad543e0...
1,Converse,[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'CO/45/3I/00/5K/11/CO453I005-K11@9.1...,DAY PACK - Sac à dos - navy,False,False,False,False,"29,95 €","24,00 €",accessoires,[One Size],CO453I005-K11,converse-sac-a-dos-navy-co453i005-k11
2,adidas Originals,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'AD/11/6D/0B/BG/11/AD116D0BB-G11@11....,GAZELLE - Baskets basses - footwear white/coll...,False,False,False,False,"59,95 €","41,95 €",shoe,"[28, 29, 30, 31, 32, 33, 34, 35]",AD116D0BB-G11,adidas-originals-gazelle-baskets-basses-ad116d...
3,s.Oliver,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'SO/22/3F/0E/TJ/11/SO223F0ET-J11@2.j...,Robe en jersey - light pink,False,False,False,False,"13,99 €","9,80 €",clothing,"[3axREG, 4axREG, 5axREG, 6axREG, 7axREG, 8axREG]",SO223F0ET-J11,soliver-robe-en-jersey-light-pink-so223f0et-j11
4,New Look 915 Generation,[],"[{'key': 'discountRate', 'value': '-35%', 'tra...",False,[{'path': 'NL/62/3A/02/VK/11/NL623A02V-K11@7.j...,PHILIIP - Jeans Skinny - mid blue,False,False,False,False,"27,95 €","18,15 €",clothing,"[9a, 10a, 11a, 12a, 13a, 14a, 15a]",NL623A02V-K11,new-look-915-generation-philiip-jeans-skinny-m...
5,Birkenstock,[],"[{'key': 'discountRate', 'value': '-25%', 'tra...",False,[{'path': 'BI/11/4G/00/AQ/11/BI114G00A-Q11@11....,NEW YORK - Sandales - black,False,False,False,False,"54,95 €","40,95 €",shoe,"[24, 26, 27, 28, 29, 30, 31, 32, 33, 34]",BI114G00A-Q11,birkenstock-new-york-sandales-bi114g00a-q11
6,Tommy Hilfiger,[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",True,[{'path': 'TO/12/4K/04/0H/11/TO124K040-H11@14....,LOGO - Sweatshirt - orange,False,True,True,False,"54,95 €","46,45 €",clothing,"[3a, 4a, 5a, 6a, 8a, 10a, 12a, 14a, 16a]",TO124K040-H11,tommy-hilfiger-logo-sweatshirt-to124k040-h11
7,The North Face,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'TH/34/3F/01/LQ/12/TH343F01L-Q12@7.j...,ANDES JACKET - Doudoune - black,False,False,False,False,"99,95 €","69,95 €",clothing,"[10a/12a, 12a/14a, 16a/18a, 5a/6a, 7a/8a]",TH343F01L-Q12,the-north-face-b-andes-jacket-doudoune-th343f0...
8,adidas Originals,[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",False,[{'path': 'AD/12/4G/02/NJ/11/AD124G02N-J11@5.j...,3 STRIPES TEE - T-shirt imprimé - real pink/white,False,False,False,False,"22,95 €","19,50 €",clothing,"[3-4a, 4-5a, 5-6a, 6-7a, 7-8a]",AD124G02N-J11,adidas-originals-3-stripes-tee-t-shirt-imprime...
9,Nike Sportswear,[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'NI/11/4D/03/TQ/11/NI114D03T-Q11@12....,COURT BOROUGH - Baskets basses - black,False,False,False,False,"44,95 €","35,95 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40]",NI114D03T-Q11,nike-sportswear-court-borough-baskets-basses-n...


#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [40]:
total_pages=results['pagination']['page_count']
df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    try:
        flattened_data1=flattened_data1.set_index('sku')
    except:
        continue
    df = df.append(flattened_data1)
df

,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,url_key
sku,,,,,,,,,,,,,,,
AD543E0Q6-Q11,adidas Performance,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -20...",False,[{'path': 'AD/54/3E/0Q/6Q/11/AD543E0Q6-Q11@4.j...,TRAINING EQUIPMENT 3 STREIFEN - Collants - bla...,False,True,True,False,"27,95 €","22,35 €",clothing,"[4-5a, 5-6a, 6-7a, 7-8a, 9-10a, 11-12a, 12-13a...",adidas-performance-collants-blackwhite-ad543e0...
CO453I005-K11,Converse,[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'CO/45/3I/00/5K/11/CO453I005-K11@9.1...,DAY PACK - Sac à dos - navy,False,False,False,False,"29,95 €","24,00 €",accessoires,[One Size],converse-sac-a-dos-navy-co453i005-k11
AD116D0BB-G11,adidas Originals,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'AD/11/6D/0B/BG/11/AD116D0BB-G11@11....,GAZELLE - Baskets basses - footwear white/coll...,False,False,False,False,"59,95 €","41,95 €",shoe,"[28, 29, 30, 31, 32, 33, 34, 35]",adidas-originals-gazelle-baskets-basses-ad116d...
SO223F0ET-J11,s.Oliver,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'SO/22/3F/0E/TJ/11/SO223F0ET-J11@2.j...,Robe en jersey - light pink,False,False,False,False,"13,99 €","9,80 €",clothing,"[3axREG, 4axREG, 5axREG, 6axREG, 7axREG, 8axREG]",soliver-robe-en-jersey-light-pink-so223f0et-j11
NL623A02V-K11,New Look 915 Generation,[],"[{'key': 'discountRate', 'value': '-35%', 'tra...",False,[{'path': 'NL/62/3A/02/VK/11/NL623A02V-K11@7.j...,PHILIIP - Jeans Skinny - mid blue,False,False,False,False,"27,95 €","18,15 €",clothing,"[9a, 10a, 11a, 12a, 13a, 14a, 15a]",new-look-915-generation-philiip-jeans-skinny-m...
BI114G00A-Q11,Birkenstock,[],"[{'key': 'discountRate', 'value': '-25%', 'tra...",False,[{'path': 'BI/11/4G/00/AQ/11/BI114G00A-Q11@11....,NEW YORK - Sandales - black,False,False,False,False,"54,95 €","40,95 €",shoe,"[24, 26, 27, 28, 29, 30, 31, 32, 33, 34]",birkenstock-new-york-sandales-bi114g00a-q11
TO124K040-H11,Tommy Hilfiger,[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",True,[{'path': 'TO/12/4K/04/0H/11/TO124K040-H11@14....,LOGO - Sweatshirt - orange,False,True,True,False,"54,95 €","46,45 €",clothing,"[3a, 4a, 5a, 6a, 8a, 10a, 12a, 14a, 16a]",tommy-hilfiger-logo-sweatshirt-to124k040-h11
TH343F01L-Q12,The North Face,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'TH/34/3F/01/LQ/12/TH343F01L-Q12@7.j...,ANDES JACKET - Doudoune - black,False,False,False,False,"99,95 €","69,95 €",clothing,"[10a/12a, 12a/14a, 16a/18a, 5a/6a, 7a/8a]",the-north-face-b-andes-jacket-doudoune-th343f0...
AD124G02N-J11,adidas Originals,[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",False,[{'path': 'AD/12/4G/02/NJ/11/AD124G02N-J11@5.j...,3 STRIPES TEE - T-shirt imprimé - real pink/white,False,False,False,False,"22,95 €","19,50 €",clothing,"[3-4a, 4-5a, 5-6a, 6-7a, 7-8a]",adidas-originals-3-stripes-tee-t-shirt-imprime...


#### Display the trending brand in DataFrame

In [41]:
df.brand_name.value_counts().index[0]

'adidas Originals'

#### Display the brand with maximal total discount (sum of discounts on all goods)

In [42]:
#Our data is still text. Convert prices into numbers:
df['price.original']=df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional']=df['price.promotional'].str.extract('(\d*,\d*)')

df['price.original'] = [x.replace(',', '.') for x in df['price.original']]
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]

In [43]:
df['discount_amount']=df['price.original'].astype(float)-df['price.promotional'].astype(float)
df1=df.copy()

In [44]:
total_disc=df1.groupby(['brand_name']).sum().discount_amount

In [45]:
total_disc.sort_values(ascending=False).index[0]

'Nike Sportswear'

#### Display the brands without discount at all

In [46]:
total_disc[total_disc==0]

Series([], Name: discount_amount, dtype: float64)